# Visual image search
_**Using a Convolutional Neural Net and OpenSearch k-Nearest Neighbors Index to retrieve visually similar images**_

---

---

## Contents


1. [Background](#Background)
1. [Setup](#Setup)
1. [TensorFlow Model Preparation](#TensorFlow-Model-Preparation)
1. [SageMaker Model Hosting](#SageMaker-Model-Hosting)
1. [Build a KNN Index in OpenSearch](#Build-a-KNN-Index-in-OpenSearch)
1. [Evaluate Index Search Results](#Evaluate-Index-Search-Results)
1. [Demo application](#Configuring-our-full-stack-visual-search-application)
1. [Extensions](#Extensions)
1. [Cleanup](#Cleanup)

## Background

In this notebook, we'll build the core components of a visual image search application. Visual image search is used in interfaces where instead of asking for something by voice or text, you show a photographic example of what you are looking for.

One of the core components of visual image search is a convolutional neural net (CNN) model that generates “feature vectors” representing both a query image and the reference item images to be compared against the query. The reference item feature vectors typically are generated offline and must be stored in a database of some sort, so they can be efficiently searched. For small reference item datasets, it is possible to use a brute force search that compares the query against every reference item. However, this is not feasible for large data sets where brute force search would become prohibitively slow. 

To enable efficient searches for visually similar images, we'll use Amazon SageMaker to generate “feature vectors” from images and use KNN algorithm in Amazon OpenSearch Service. KNN for Amazon OpenSearch Service lets you search for points in a vector space and find the "nearest neighbors" for those points by Euclidean distance or cosine similarity(default is Euclidean distance). Use cases include recommendations (for example, an "other songs you might like" feature in a music application), image recognition, and fraud detection.

Here are the steps we'll follow to build the visual image search: After some initial setup, we'll prepare a model using TensorFlow for generating feature vectors, then generate feature vectors of Fashion Images from *__feidegger__*, a *__zalandoresearch__* dataset. Those feature vectors will be imported in Amazon Elasticsearch KNN Index. Next, we'll explore some test image queries, and visualize the results.


## Setup

In [ ]:
#Install tqdm to have progress bar
!pip install tqdm

#install necessary pkg to make connection with opensearch domain
!pip install opensearch-py
!pip install requests
!pip install retry

In [ ]:
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

In [ ]:
cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "VisualSearchEngineInfraStack"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['S3TrainingBucketOutput']
oss_host = outputs['OpenSearchHostName']
backend_lambda_arn = outputs["PostFetchSimilarPhotosLambda"]
api = outputs["ImageSimilarityApi"]
website_url = outputs["S3WebsiteURLOutput"]

outputs

## Zalando Research data

The dataset itself consists of 8732 high-resolution images, each depicting a dress from the available on the Zalando shop against a white-background. 

**Downloading Zalando Research data** - https://github.com/zalandoresearch/feidegger

### For simplicity, we've already downloaded and copied it under the S3 Training Bucket. You can explore it using the S3 console if you wish!

 **Citation:** <br>
 *@inproceedings{lefakis2018feidegger,* <br>
 *title={FEIDEGGER: A Multi-modal Corpus of Fashion Images and Descriptions in German},* <br>
 *author={Lefakis, Leonidas and Akbik, Alan and Vollgraf, Roland},* <br>
 *booktitle = {{LREC} 2018, 11th Language Resources and Evaluation Conference},* <br>
 *year      = {2018}* <br>
 *}*

## TensorFlow Model Preparation

We'll use TensorFlow backend to prepare a model for "featurizing" images into feature vectors. TensorFlow has a native Module API, as well as a higher level Keras API. 

We will start with a pretrained model, avoiding spending time and money training a model from scratch. Accordingly, as a first step in preparing the model, we'll import a pretrained model from Keras application. Researchers have experimented with various pretrained CNN architectures with different numbers of layers, discovering that there are several good possibilities.

In this notebook, we'll select a model based on the ResNet architecture, a commonly used choice. Of the various choices for number of layers, ranging from 18 to 152, we'll use 50 layers. This also is a common choice that balances the expressiveness of the resulting feature vectors (embeddings) against computational efficiency (lower number of layers means greater efficiency at the cost of less expressiveness).

In [ ]:
import os
import json
import time
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import sagemaker
from PIL import Image
from sagemaker.tensorflow import TensorFlow

In [ ]:
# Set the channel first for better performance
from tensorflow.keras import backend
backend.set_image_data_format('channels_first')
print(backend.image_data_format())

Now we'll get a reference Xception model which is trained on Imagenet dataset to extract the feature without the actual classifier. Xception is a convolutional neural network that is 71 layers deep. More specifically, we'll use that layer to generate a row vector of floating point numbers as an "embedding" or representation of the features of the image. We'll also save the model as *SavedModel* format under **export/Servo/1** to serve from SageMaker TensorFlow serving API.

In [ ]:
# Import Xception model
model = tf.keras.applications.Xception(weights='imagenet', include_top=False,input_shape=(3, 224, 224),pooling='avg')


In [ ]:
model.summary()

In [ ]:
# Creating the directory strcture with model version
dirName = 'export/Xception/1'
if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")    

In [ ]:
# Save the model in SavedModel format
model.save('./export/Xception/1/', save_format='tf')

In [ ]:
# Check the model Signature
!saved_model_cli show --dir ./export/Xception/1/ --tag_set serve --signature_def serving_default

## SageMaker Model Hosting

After saving the feature extractor model we will deploy the model using Sagemaker Tensorflow Serving api which is a flexible, high-performance serving system for machine learning models, designed for production environments.TensorFlow Serving makes it easy to deploy new algorithms and experiments, while keeping the same server architecture and APIs. TensorFlow Serving provides out-of-the-box integration with TensorFlow models, but can be easily extended to serve other types of models and data. We will define **inference.py** to customize the input data to TensorFlow serving API. We also need to add **requirements.txt** file for aditional libraby in the tensorflow serving container.

In [ ]:
# check the actual content of inference.py
!pygmentize src/inference.py

In [ ]:
import tarfile

# zip the model .gz format
model_version = '1'
export_dir = 'export/Xception/' + model_version
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
# Upload the model to S3
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')
inputs

After we upload the model to S3 we will use TensorFlow serving container to host the model. We are using ml.m5.xlarge instance type with gpu acceleration of ml.eia1.medium. We will use this endpoint to generate features and import into ElasticSearch. You can also choose to use p3 type gpu instances for better performance.

In [ ]:
# Deploy the model in Sagemaker Endpoint. This process will take ~10 min. We'll be using GPU accelerated inference.

from sagemaker.tensorflow import TensorFlowModel

sagemaker_model = TensorFlowModel(entry_point='inference.py', model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role, framework_version='2.3', source_dir='./src' )

predictor = sagemaker_model.deploy(initial_instance_count=5, instance_type='ml.m5.xlarge', accelerator_type="ml.eia1.medium")


In [ ]:
%%time
# get the features for a sample image
from sagemaker.serializers import IdentitySerializer
payload = s3.get_object(Bucket=bucket,Key='data/feidegger/fashion/0000723855b24fbe806c20a1abd9d5dc.jpg')['Body'].read()
predictor.serializer = IdentitySerializer(content_type='application/x-image')
features = predictor.predict(payload)['predictions'][0]

# Here are the features extracted from the second last layer of the network
features


## Build a KNN Index in OpenSearch

KNN for Amazon OpenSearch Service lets you search for points in a vector space and find the "nearest neighbors" for those points by Euclidean distance or cosine similarity (default is Euclidean distance). Use cases include recommendations (for example, an "other songs you might like" feature in a music application), image recognition, and fraud detection.

Full documentation for the OpenSearch feature, including descriptions of settings and statistics, is available here - https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html

In this step we'll get all the features zalando images and import those features into OpenSearch 1.2 domain.

In [ ]:
# Define some utility function

# return all s3 keys
def get_all_s3_keys(bucket):
    """Get a list of all keys in an S3 bucket."""    
    keys = []

    kwargs = {'Bucket': bucket}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            keys.append('s3://' + bucket + '/' + obj['Key'])

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return keys

In [ ]:
# get all the zalando images keys from the bucket make a list
s3_uris = get_all_s3_keys(bucket)

In [ ]:
# define a function to extract image features
from time import sleep

sm_client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME = predictor.endpoint_name

def get_predictions(payload):
    return sm_client.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                           ContentType='application/x-image',
                                           Body=payload)

def extract_features(s3_uri):
    key = s3_uri.replace(f's3://{bucket}/', '')
    payload = s3.get_object(Bucket=bucket,Key=key)['Body'].read()
    try:
        response = get_predictions(payload)
    except:
        sleep(0.1)
        response = get_predictions(payload)

    del payload
    response_body = json.loads((response['Body'].read()))
    feature_lst = response_body['predictions'][0]
    
    return key, feature_lst


*** Next step will take ~ 5-10 min. It will extract the features using the sagemaker inference endpoint from all the images in our training bucket. You can choose to skip this step and move directly to the next one by using a saved pickle file. ***

In [ ]:
# This process cell will take approximately 24-25 minutes on a t3.medium notebook instance
# with 3 m5.xlarge SageMaker Hosted Endpoint instances
from multiprocessing import cpu_count
from tqdm.contrib.concurrent import process_map
import pickle

workers = 2 * cpu_count()
extraction_result = process_map(extract_features, s3_uris, max_workers=workers)
extraction_result_file = open('extraction_result_file.txt', 'wb')
pickle.dump(extraction_result, extraction_result_file)
extraction_result_file.close()

If you skipped the last step, please execute the next cell to unpickle the extraction results from the saved file:

In [ ]:
import pickle

extraction_result_file = open('extraction_result_file.txt', 'rb')
extraction_result = pickle.load(extraction_result_file)
extraction_result_file.close()

### Creating an OpenSearch KNN index and loading our extracted features for all the images into that

In [ ]:
# setting up the Elasticsearch connection
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
region = boto3.Session().region_name # e.g. us-east-1
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)

oss = OpenSearch(
    hosts = [{'host': oss_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
# Define KNN OpenSearch index maping
knn_index = {
    "settings": {
        "index.knn": True
    },
    "mappings": {
        "properties": {
            "zalando_img_vector": {
                "type": "knn_vector",
                "dimension": 2048
            }
        }
    }
}

In [ ]:
# Creating the OpenSearch index
oss.indices.create(index="idx_zalando",body=knn_index,ignore=400)
oss.indices.get(index="idx_zalando")

In [ ]:
# defining a function to import the feature vectors corresponds to each S3 URI into OpenSearch KNN index
# This process will take around ~3 min.


def oss_import(i):
    oss.index(index='idx_zalando',
             body={"zalando_img_vector": i[1], 
                   "image": i[0]}
            )
    
result = process_map(oss_import, extraction_result, max_workers=workers)

## Evaluate Index Search Results

In this step we will use SageMaker SDK as well as Boto3 SDK to query the OpenSearch to retrive the nearest neighbours. One thing to mention **zalando** dataset has pretty good similarity with Imagenet dataset. Now if you hav a very domain specific problem then then you need to train that dataset on top of pretrained feature extractor model such as VGG, Resnet, Mobilenet etc and build a new feature extractor model.

In [ ]:
# define display_image function
def display_image(bucket, key):
    response = s3.get_object(Bucket=bucket,Key=key)['Body']
    img = Image.open(response)
    return display(img)

In [ ]:
import requests
import random
from PIL import Image
import io
url = "https://www.77onlineshop.eu/$WS/77onlineshop/websale8_shop-77onlineshop/produkte/medien/bilder/gross/21031469.jpg"

img_bytes = requests.get(url).content
query_img = Image.open(io.BytesIO(img_bytes))
query_img

In [ ]:
# Get image features through the SageMaker SDK by calling our SageMaker inference endpoint
from sagemaker.serializers import IdentitySerializer
predictor.serializer = IdentitySerializer(content_type='application/x-image')
features = predictor.predict(img_bytes)['predictions'][0]

In [ ]:
import json
k = 3 # Set to the number of nearest matches
idx_name = 'idx_zalando'
res = oss.search(request_timeout=30, index=idx_name,
                body={'size': k, 
                      'query': {'knn': {'zalando_img_vector': {'vector': features, 'k': k}}}})

In [ ]:
for i in range(k):
    key = res['hits']['hits'][i]['_source']['image']
    img = display_image(bucket,key)

# Configuring our full-stack visual search application

Updating our lambda function with sagemaker endpoint created during the lab

In [ ]:
lambda_client = boto3.client('lambda')
response = lambda_client.update_function_configuration(
    FunctionName=backend_lambda_arn,
    Environment={
        "Variables": {
            "SM_ENDPOINT": ENDPOINT_NAME,
            "OSS_ENDPOINT": oss_host,
            "S3_TRAINING_BUCKET": bucket
        }
    }
)

Now that you have a working Amazon SageMaker endpoint for extracting image features and a KNN index on OpenSearch, you are ready to build a real-world full-stack ML-powered web app. Amazon API Gateway and AWS Lambda function were created as part of the infra setup before the lab using cdk. The Lambda function runs your code in response to HTTP requests that are sent to the API Gateway.

In [ ]:
# Review the content of the Lambda function code.
!pygmentize backend/index.py

### And that's it, you're ready to play around with the app

In [ ]:
print('Click the URL below:\n')
print(website_url + '/index.html')

You should see the following page:

![Website](./images/pi3small.png)

On the website, try pasting the following URL in the URL text field.

`https://www.77onlineshop.eu/$WS/77onlineshop/websale8_shop-77onlineshop/produkte/medien/bilder/gross/21031469.jpg`

## Extensions

We have used pretrained Xception model which is trained on Imagenet dataset. Now based on your use-case you can fine tune any pre-trained models, such as VGG, Resnet, and MobileNet with your own dataset and host the model in Amazon SageMaker.

You can also use Amazon SageMaker Batch transform job to have a bulk feaures extracted from your stored S3 images and then you can use AWS Glue to import that data into Elasticeearch domain.


### Cleanup

Make sure that you delete the Amazon SageMaker endpoint and delete the features form the Elasticsearch domain to prevent any additional charges.

In [ ]:
# Delete the endpoint
predictor.delete_endpoint()

In [ ]:
# Delete the OpenSearch index that holds the image features
oss.indices.delete(index = "idx_zalando")